In [1]:
import argparse
import pandas as pd
from tqdm import tqdm
import yaml
import math
import numpy as np
from datetime import datetime
import json
import bisect

config = yaml.load(open('../config/config.yaml'))
exception_name_columns = config['exception_name']['train']

<ipython-input-1-e16a28c1aa31>:11: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open('../config/config.yaml'))


In [ ]:
n_exceptions_train = pd.read_csv('../'+config['data']['feature']['train']['nc_exceptions_key_ip_column_name_value_timelist'])
# test sample 里所有nc 都不在test exception里面出现，因此不需要nc_exceptions_test的信息。
n_exceptions_test = pd.read_csv('../'+config['data']['process']['test']['nc_exceptions_key_ip_column_name_value_timelist'])
